In [ ]:
!dir ..\data

In [1]:
import pandas as pd
import numpy as np

In [2]:
root = "../data/"
game = root+'game.csv'
small_game = root+'game_small.csv'
vs_game = root+'game_very_small.csv'

gt = root+'game_teams_stats.csv'
small_gt = root+'game_teams_stats_small.csv'
vs_gt = root+ 'game_teams_stats_very_small.csv'

team = root+'team_info.csv'

team_df = pd.read_csv(team)
team_df.index = team_df.team_id
team_ser = team_df['teamName']
team_dict = team_ser.to_dict()

game_df = pd.read_csv(game)
small_game_df = pd.read_csv(small_game)
vs_game_df = pd.read_csv(vs_game)

for df in [game_df,small_game_df, vs_game_df]:
    df['homeTeam'] = df['home_team_id'].map(team_dict)
    df['awayTeam'] = df['away_team_id'].map(team_dict)
    
gt_df = pd.read_csv(gt)
small_gt_df = pd.read_csv(small_gt)
vs_gt_df = pd.read_csv(vs_gt)



In [12]:
vs_game_df

,Unnamed: 0,game_id,season,type,date_time,away_team_id,home_team_id,away_goals,home_goals,outcome,home_rink_side_start,venue,venue_link,venue_time_zone_id,venue_time_zone_offset,venue_time_zone_tz,homeTeam,awayTeam
0,0,2012030221,20122013,P,2013-05-16,3,6,2,3,home win OT,left,TD Garden,/api/v1/venues/null,America/New_York,-4,EDT,Bruins,Rangers
1,1,2012030222,20122013,P,2013-05-19,3,6,2,5,home win REG,left,TD Garden,/api/v1/venues/null,America/New_York,-4,EDT,Bruins,Rangers


In [3]:
to_keep = ['game_id', 'team_id', 'HoA', 'won', 'settled_in', 'head_coach', 'goals',
       'shots', 'hits', 'pim', 'powerPlayOpportunities', 'powerPlayGoals',
       'faceOffWinPercentage', 'giveaways', 'takeaways', 'season', 'type',
       'date_time','venue_time_zone_tz','teamName', "home_goals","away_goals", 'homeTeam','awayTeam']

In [4]:
# gt_df.merge(game_df, on = "game_id"
#            ).merge(team_df, on = "team_id"
#            ).merge(team_df[['team_id','teamName']],
#                    left_on = "home_team_id", right_on = "team_id",
#                    suffixes = ("Home","_")
#            ).merge(team_df[['team_id','teamName']],
#                    left_on = "away_team_id", right_on = "team_id",
#                    suffixes = ("Away","_"))


In [9]:
full_df = gt_df.merge(game_df, on = "game_id").merge(team_df, on = "team_id")[to_keep]
small_df = small_gt_df.merge(small_game_df, on = "game_id").merge(team_df, on = "team_id")[to_keep]
vs_df = vs_gt_df.merge(vs_game_df, on = "game_id").merge(team_df, on = "team_id")[to_keep]

df_names = ['full','small', 'vs']
dfs = [full_df,small_df, vs_df]

for df in dfs:
    df['opp_team'] = np.where(df['teamName'] == df['homeTeam'], df['awayTeam'],df['homeTeam'])
    df['opp_goal'] = np.where(df.HoA == "away", df.home_goals, df.away_goals)
    df['margin'] = df['goals'] - df['opp_goal']

C:\Users\William\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: 'team_id' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  """Entry point for launching an IPython kernel.
C:\Users\William\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: 'team_id' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  
C:\Users\William\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: 'team_id' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  This is separate from the ipykernel package so we can avoid doing imports until


In [10]:
full_df.head()

,game_id,team_id,HoA,won,settled_in,head_coach,goals,shots,hits,pim,...,date_time,venue_time_zone_tz,teamName,home_goals,away_goals,homeTeam,awayTeam,opp_team,opp_goal,margin
0,2012030221,3,away,False,OT,John Tortorella,2,35,44,8,...,2013-05-16,EDT,Rangers,3,2,Bruins,Rangers,Bruins,3,-1
1,2012030222,3,away,False,REG,John Tortorella,2,37,33,11,...,2013-05-19,EDT,Rangers,5,2,Bruins,Rangers,Bruins,5,-3
2,2012030223,3,home,False,REG,John Tortorella,1,24,37,2,...,2013-05-21,EDT,Rangers,1,2,Rangers,Bruins,Bruins,2,-1
3,2012030224,3,home,True,OT,John Tortorella,4,32,40,8,...,2013-05-23,EDT,Rangers,4,3,Rangers,Bruins,Bruins,3,1
4,2012030225,3,away,False,REG,John Tortorella,1,29,25,13,...,2013-05-25,EDT,Rangers,3,1,Bruins,Rangers,Bruins,3,-2


### highest_total_score

In [8]:
for name, df in zip(df_names, dfs):
    print(name,"\n", df.groupby('game_id')['goals'].sum().sort_values(ascending = False).head(1), '\n')

full 
 game_id
2016020661    15
Name: goals, dtype: int64 

small 
 game_id
2012030312    7
Name: goals, dtype: int64 

vs 
 game_id
2012030222    7
Name: goals, dtype: int64 



### lowest_total_score

In [9]:
for name, df in zip(df_names, dfs):
    print(name,"\n", df.groupby('game_id')['goals'].sum().sort_values(ascending = True).head(1), '\n')

full 
 game_id
2014020456    0
Name: goals, dtype: int64 

small 
 game_id
2012030314    1
Name: goals, dtype: int64 

vs 
 game_id
2012030221    5
Name: goals, dtype: int64 



### biggest_blowout

In [10]:
def find_diff(x):
    if len(x) == 1: return 0
    else: return abs(x[0]-x[1])
    
    
for name, df in zip(df_names, dfs):
    print(name,"\n",df.groupby('game_id')['goals'].unique().apply(find_diff).sort_values(ascending = False).head(1), "\n")

full 
 game_id
2016020158    10
Name: goals, dtype: int64 

small 
 game_id
2012030312    5
Name: goals, dtype: int64 

vs 
 game_id
2012030222    3
Name: goals, dtype: int64 



### count_of_games_by_season

In [11]:
for name, df in zip(df_names, dfs):
    print(name,"\n",df.groupby('season')['goals'].count()/2, "\n")

full 
 season
20122013     806.0
20132014    1323.0
20142015    1319.0
20152016    1321.0
20162017    1317.0
20172018    1355.0
Name: goals, dtype: float64 

small 
 season
20122013    20.0
Name: goals, dtype: float64 

vs 
 season
20122013    2.0
Name: goals, dtype: float64 



### percentage_visitor_wins

In [12]:
for name, df in zip(df_names, dfs):
    print(name,"\n", df[df.HoA == "away"].won.mean(),"\n")

full 
 0.4504770864131165 

small 
 0.3 

vs 
 0.0 



### percentage_home_wins

In [13]:
for name, df in zip(df_names, dfs):
    print(name,"\n", df[df.HoA == "home"].won.mean(),"\n")

full 
 0.5495229135868834 

small 
 0.7 

vs 
 1.0 



 ### average_goals_per_game

In [14]:
for name, df in zip(df_names, dfs):
    print(name,"\n", df.groupby('game_id')['goals'].sum().mean(),"\n")

full 
 5.44080096761188 

small 
 4.65 

vs 
 6.0 



### average_goals_by_season

In [15]:
for name, df in zip(df_names, dfs):
    print(name,"\n",df.groupby('season')['goals'].sum() /( df.groupby('season')['goals'].count()/2), "\n")

full 
 season
20122013    5.277916
20132014    5.361300
20142015    5.304776
20152016    5.331567
20162017    5.431283
20172018    5.863469
Name: goals, dtype: float64 

small 
 season
20122013    4.65
Name: goals, dtype: float64 

vs 
 season
20122013    6.0
Name: goals, dtype: float64 



# League Stats

### count_of_teams

In [16]:
for name, df in zip(df_names, dfs):
    print(name,"\n",df['team_id'].unique().shape[0],"\n")

full 
 32 

small 
 7 

vs 
 2 



### best_offense

In [17]:
for name, df in zip(df_names, dfs):
    print(name,"\n", df.groupby('teamName')['goals'].mean().sort_values(ascending = False).head(1), "\n")

full 
 teamName
Golden Knights    3.186275
Name: goals, dtype: float64 

small 
 teamName
Senators    3.5
Name: goals, dtype: float64 

vs 
 teamName
Bruins    4
Name: goals, dtype: int64 



### worst_offense

In [18]:
for name, df in zip(df_names, dfs):
    print(name,"\n", df.groupby('teamName')['goals'].mean().sort_values(ascending = True).head(1), "\n")

full 
 teamName
Sabres    2.220524
Name: goals, dtype: float64 

small 
 teamName
Penguins    0.5
Name: goals, dtype: float64 

vs 
 teamName
Rangers    2
Name: goals, dtype: int64 



### worst_defense

In [19]:
for df in dfs:
    

In [20]:
vs_df

,game_id,team_id,HoA,won,settled_in,head_coach,goals,shots,hits,pim,...,giveaways,takeaways,season,type,date_time,venue_time_zone_tz,teamName,home_goals,away_goals,opp_goal
0,2012030221,3,away,False,OT,John Tortorella,2,35,44,8,...,17,7,20122013,P,2013-05-16,EDT,Rangers,3,2,3
1,2012030222,3,away,False,REG,John Tortorella,2,37,33,11,...,1,4,20122013,P,2013-05-19,EDT,Rangers,5,2,5
2,2012030221,6,home,True,OT,Claude Julien,3,48,51,6,...,4,5,20122013,P,2013-05-16,EDT,Bruins,3,2,2
3,2012030222,6,home,True,REG,Claude Julien,5,32,36,19,...,16,6,20122013,P,2013-05-19,EDT,Bruins,5,2,2


In [21]:
for name, df in zip(df_names, dfs):
    print(name,"\n", df.groupby('teamName')['opp_goal'].mean().sort_values(ascending = False).head(1), "\n")

full 
 teamName
Sabres    3.065502
Name: opp_goal, dtype: float64 

small 
 teamName
Canadiens    3.5
Name: opp_goal, dtype: float64 

vs 
 teamName
Rangers    4
Name: opp_goal, dtype: int64 



### best_defense

In [22]:
for name, df in zip(df_names, dfs):
    print(name,"\n", df.groupby('teamName')['opp_goal'].mean().sort_values(ascending = True).head(1), "\n")

full 
 teamName
Kings    2.401174
Name: opp_goal, dtype: float64 

small 
 teamName
Bruins    1.333333
Name: opp_goal, dtype: float64 

vs 
 teamName
Bruins    2
Name: opp_goal, dtype: int64 



### highest_scoring_home_team

In [23]:
for name, df in zip(df_names, dfs):
    print(name,"\n", df[df.HoA == "home"].groupby("teamName")['goals'].mean().sort_values(ascending = False).head(1), "\n") 

full 
 teamName
Golden Knights    3.568627
Name: goals, dtype: float64 

small 
 teamName
Senators    4.5
Name: goals, dtype: float64 

vs 
 teamName
Bruins    4
Name: goals, dtype: int64 



### lowest_scoring_home_team

In [24]:
for name, df in zip(df_names, dfs):
    print(name,"\n", df[df.HoA == "home"].groupby("teamName")['goals'].mean().sort_values(ascending = True).head(1), "\n") 

full 
 teamName
Sabres    2.248908
Name: goals, dtype: float64 

small 
 teamName
Penguins    0.5
Name: goals, dtype: float64 

vs 
 teamName
Bruins    4
Name: goals, dtype: int64 



### highest_scoring_visitor

In [25]:
for name, df in zip(df_names, dfs):
    print(name,"\n", df[df.HoA == "away"].groupby("teamName")['goals'].mean().sort_values(ascending = False).head(1), "\n") 

full 
 teamName
Capitals    2.901515
Name: goals, dtype: float64 

small 
 teamName
Bruins    3.5
Name: goals, dtype: float64 

vs 
 teamName
Rangers    2
Name: goals, dtype: int64 



### lowest_scoring_visitor

In [26]:
for name, df in zip(df_names, dfs):
    print(name,"\n", df[df.HoA == "away"].groupby("teamName")['goals'].mean().sort_values(ascending = True).head(1), "\n") 

full 
 teamName
Sabres    2.19214
Name: goals, dtype: float64 

small 
 teamName
Penguins    0.5
Name: goals, dtype: float64 

vs 
 teamName
Rangers    2
Name: goals, dtype: int64 



### winnigest_team

In [27]:
for name, df in zip(df_names, dfs):
    print(name,"\n", df.groupby("teamName")['won'].mean().sort_values(ascending = False).head(1), "\n")

full 
 teamName
Golden Knights    0.627451
Name: won, dtype: float64 

small 
 teamName
Bruins    0.888889
Name: won, dtype: float64 

vs 
 teamName
Bruins    True
Name: won, dtype: bool 



### best_fans

In [28]:
for name, df in zip(df_names, dfs):
    home_win = df[df.HoA == "home"].groupby("teamName")['won'].mean().sort_index()
    away_win = df[df.HoA == "away"].groupby("teamName")['won'].mean().sort_index()
    
    print(name, "\n", (home_win-away_win).sort_values(ascending = False).head(1), "\n")

full 
 teamName
Flyers    0.18229
Name: won, dtype: float64 

small 
 teamName
Senators    0.5
Name: won, dtype: float64 

vs 
 teamName
Bruins    NaN
Name: won, dtype: object 



### worst_fans

**should be empty arrays**

In [29]:
for name, df in zip(df_names, dfs):
    home_win = df[df.HoA == "home"].groupby("teamName")['won'].mean().sort_index()
    away_win = df[df.HoA == "away"].groupby("teamName")['won'].mean().sort_index()
#     print(home_win)
#     print(away_win)
    home_minus_away = (home_win-away_win)
    
    print(name, "\n", home_minus_away[home_minus_away <0], "\n")

full 
 Series([], Name: won, dtype: float64) 

small 
 Series([], Name: won, dtype: float64) 

vs 
 Series([], Name: won, dtype: object) 



# Team Statistics

### team_info

In [36]:
for name, df in zip(df_names, dfs):
    print(name,"\n", pd.concat([df.groupby("teamName")['head_coach'].unique(),df.groupby("teamName")['teamName'].unique()], axis = 1).to_dict(), "/n")

full 
 {'head_coach': {'Avalanche': array(['Patrick Roy', 'Jared Bednar', 'Joe Sacco'], dtype=object), 'Blackhawks': array(['Joel Quenneville'], dtype=object), 'Blue Jackets': array(['John Tortorella', 'Todd Richards'], dtype=object), 'Blues': array(['Ken Hitchcock', 'Mike Yeo'], dtype=object), 'Bruins': array(['Claude Julien', 'Bruce Cassidy'], dtype=object), 'Canadiens': array(['Michel Therrien', 'Claude Julien'], dtype=object), 'Canucks': array(['John Tortorella', 'Alain Vigneault', 'Willie Desjardins',
       'Travis Green'], dtype=object), 'Capitals': array(['Adam Oates', 'Barry Trotz'], dtype=object), 'Coyotes': array(['Dave Tippett', 'Rick Tocchet'], dtype=object), 'Devils': array(['John Hynes', 'Peter DeBoer', 'Adam Oates'], dtype=object), 'Ducks': array(['Bruce Boudreau', 'Randy Carlyle'], dtype=object), 'Flames': array(['Glen Gulutzan', 'Bob Hartley'], dtype=object), 'Flyers': array(['Dave Hakstol', 'Craig Berube', 'Peter Laviolette'], dtype=object), 'Golden Knights': array([

### best_season

In [37]:
for name, df in zip(df_names, dfs):
    data = df.groupby(["teamName", "season"])["won"].mean().unstack().T
    
    print(name,"\n")
    for col in data:
        print("teamName:", col, "best season", data[col].sort_values(ascending = False).head(1).index[0])
          
    print("\n")

full 

teamName: Avalanche best season 20132014
teamName: Blackhawks best season 20122013
teamName: Blue Jackets best season 20162017
teamName: Blues best season 20132014
teamName: Bruins best season 20132014
teamName: Canadiens best season 20142015
teamName: Canucks best season 20142015
teamName: Capitals best season 20152016
teamName: Coyotes best season 20132014
teamName: Devils best season 20172018
teamName: Ducks best season 20132014
teamName: Flames best season 20142015
teamName: Flyers best season 20132014
teamName: Golden Knights best season 20172018
teamName: Hurricanes best season 20172018
teamName: Islanders best season 20142015
teamName: Jets best season 20172018
teamName: Kings best season 20132014
teamName: Lightning best season 20172018
teamName: Maple Leafs best season 20172018
teamName: Oilers best season 20162017
teamName: Panthers best season 20152016
teamName: Penguins best season 20122013
teamName: Predators best season 20172018
teamName: Rangers best season 201420

### worst_season

In [38]:
for name, df in zip(df_names, dfs):
    data = df.groupby(["teamName", "season"])["won"].mean().unstack().T
    
    print(name,"\n")
    for col in data:
        print("teamName:", col, "worst season", data[col].sort_values(ascending = True).head(1).index[0])
          
    print("\n")

full 

teamName: Avalanche worst season 20162017
teamName: Blackhawks worst season 20172018
teamName: Blue Jackets worst season 20152016
teamName: Blues worst season 20172018
teamName: Bruins worst season 20142015
teamName: Canadiens worst season 20172018
teamName: Canucks worst season 20162017
teamName: Capitals worst season 20132014
teamName: Coyotes worst season 20142015
teamName: Devils worst season 20162017
teamName: Ducks worst season 20172018
teamName: Flames worst season 20122013
teamName: Flyers worst season 20142015
teamName: Golden Knights worst season 20172018
teamName: Hurricanes worst season 20142015
teamName: Islanders worst season 20132014
teamName: Jets worst season 20152016
teamName: Kings worst season 20162017
teamName: Lightning worst season 20122013
teamName: Maple Leafs worst season 20152016
teamName: Oilers worst season 20142015
teamName: Panthers worst season 20122013
teamName: Penguins worst season 20142015
teamName: Predators worst season 20122013
teamName: Ra

### average_win_percentage

In [39]:
for name, df in zip(df_names, dfs):
    data = df.groupby(["teamName", "season"])["won"].mean().unstack().T
    
    print(name,"\n")
    for col in data:
        print("teamName:", col, "avg win pct", round(data[col].mean(),4))
          
    print("\n")

full 

teamName: Avalanche avg win pct 0.447
teamName: Blackhawks avg win pct 0.5753
teamName: Blue Jackets avg win pct 0.5097
teamName: Blues avg win pct 0.5774
teamName: Bruins avg win pct 0.5615
teamName: Canadiens avg win pct 0.5169
teamName: Canucks avg win pct 0.4382
teamName: Capitals avg win pct 0.5793
teamName: Coyotes avg win pct 0.388
teamName: Devils avg win pct 0.4225
teamName: Ducks avg win pct 0.5838
teamName: Flames avg win pct 0.4603
teamName: Flyers avg win pct 0.4752
teamName: Golden Knights avg win pct 0.6275
teamName: Hurricanes avg win pct 0.4176
teamName: Islanders avg win pct 0.4871
teamName: Jets avg win pct 0.497
teamName: Kings avg win pct 0.5282
teamName: Lightning avg win pct 0.5412
teamName: Maple Leafs avg win pct 0.462
teamName: Oilers avg win pct 0.4046
teamName: Panthers avg win pct 0.4416
teamName: Penguins avg win pct 0.5999
teamName: Predators avg win pct 0.5023
teamName: Rangers avg win pct 0.537
teamName: Red Wings avg win pct 0.4546
teamName: Sab

### most_goals_scored

In [40]:
for name, df in zip(df_names, dfs):
    data = df.groupby(["teamName", "season"])["goals"].max().unstack().T
    
    print(name,"\n")
    for col in data:
        print("teamName:", col, "most_goals", data[col].sort_values(ascending = False).iloc[0])
          
    print("\n")

full 

teamName: Avalanche most_goals 7.0
teamName: Blackhawks most_goals 10.0
teamName: Blue Jackets most_goals 10.0
teamName: Blues most_goals 8.0
teamName: Bruins most_goals 8.0
teamName: Canadiens most_goals 10.0
teamName: Canucks most_goals 7.0
teamName: Capitals most_goals 7.0
teamName: Coyotes most_goals 7.0
teamName: Devils most_goals 8.0
teamName: Ducks most_goals 9.0
teamName: Flames most_goals 8.0
teamName: Flyers most_goals 8.0
teamName: Golden Knights most_goals 7.0
teamName: Hurricanes most_goals 8.0
teamName: Islanders most_goals 8.0
teamName: Jets most_goals 8.0
teamName: Kings most_goals 9.0
teamName: Lightning most_goals 8.0
teamName: Maple Leafs most_goals 8.0
teamName: Oilers most_goals 8.0
teamName: Panthers most_goals 8.0
teamName: Penguins most_goals 8.0
teamName: Predators most_goals 9.0
teamName: Rangers most_goals 8.0
teamName: Red Wings most_goals 8.0
teamName: Sabres most_goals 7.0
teamName: Senators most_goals 7.0
teamName: Sharks most_goals 9.0
teamName: S

### fewest_goals_scored

In [41]:
for name, df in zip(df_names, dfs):
    data = df.groupby(["teamName", "season"])["goals"].min().unstack().T
    
    print(name,"\n")
    for col in data:
        print("teamName:", col, "fewest goals", data[col].sort_values(ascending = True).iloc[0])
          
    print("\n")

full 

teamName: Avalanche fewest goals 0.0
teamName: Blackhawks fewest goals 0.0
teamName: Blue Jackets fewest goals 0.0
teamName: Blues fewest goals 0.0
teamName: Bruins fewest goals 0.0
teamName: Canadiens fewest goals 0.0
teamName: Canucks fewest goals 0.0
teamName: Capitals fewest goals 0.0
teamName: Coyotes fewest goals 0.0
teamName: Devils fewest goals 0.0
teamName: Ducks fewest goals 0.0
teamName: Flames fewest goals 0.0
teamName: Flyers fewest goals 0.0
teamName: Golden Knights fewest goals 0.0
teamName: Hurricanes fewest goals 0.0
teamName: Islanders fewest goals 0.0
teamName: Jets fewest goals 0.0
teamName: Kings fewest goals 0.0
teamName: Lightning fewest goals 0.0
teamName: Maple Leafs fewest goals 0.0
teamName: Oilers fewest goals 0.0
teamName: Panthers fewest goals 0.0
teamName: Penguins fewest goals 0.0
teamName: Predators fewest goals 0.0
teamName: Rangers fewest goals 0.0
teamName: Red Wings fewest goals 0.0
teamName: Sabres fewest goals 0.0
teamName: Senators fewest 

### favorite_opponent

name of opponent with lowest win pct agains team

In [38]:
for df,name in zip(dfs,df_names):
    print(name)
    data = df.groupby(["teamName","opp_team"])["won"].mean().unstack().T

    for col in data:
        print("team_name: ", col, "fav_opp:", data[col].sort_values(ascending = False).index[0])
        
    print("\n\n")

full
team_name:  Avalanche fav_opp: Bruins
team_name:  Blackhawks fav_opp: Sabres
team_name:  Blue Jackets fav_opp: Stars
team_name:  Blues fav_opp: Sabres
team_name:  Bruins fav_opp: Coyotes
team_name:  Canadiens fav_opp: Canucks
team_name:  Canucks fav_opp: Hurricanes
team_name:  Capitals fav_opp: Bruins
team_name:  Coyotes fav_opp: Oilers
team_name:  Devils fav_opp: Wild
team_name:  Ducks fav_opp: Bruins
team_name:  Flames fav_opp: Devils
team_name:  Flyers fav_opp: Jets
team_name:  Golden Knights fav_opp: Lightning
team_name:  Hurricanes fav_opp: Coyotes
team_name:  Islanders fav_opp: Golden Knights
team_name:  Jets fav_opp: Coyotes
team_name:  Kings fav_opp: Panthers
team_name:  Lightning fav_opp: Hurricanes
team_name:  Maple Leafs fav_opp: Oilers
team_name:  Oilers fav_opp: Golden Knights
team_name:  Panthers fav_opp: Sabres
team_name:  Penguins fav_opp: Sabres
team_name:  Predators fav_opp: Oilers
team_name:  Rangers fav_opp: Coyotes
team_name:  Red Wings fav_opp: Predators
team

### rival
name of opponent with higest win pct against team

In [39]:
for df,name in zip(dfs,df_names):
    print(name)
    data = df.groupby(["teamName","opp_team"])["won"].mean().unstack().T

    for col in data:
        print("team_name: ", col, "rival:", data[col].sort_values(ascending = True).index[0])
        
    print("\n\n")

full
team_name:  Avalanche rival: Kings
team_name:  Blackhawks rival: Golden Knights
team_name:  Blue Jackets rival: Kings
team_name:  Blues rival: Capitals
team_name:  Bruins rival: Ducks
team_name:  Canadiens rival: Sharks
team_name:  Canucks rival: Golden Knights
team_name:  Capitals rival: Predators
team_name:  Coyotes rival: Bruins
team_name:  Devils rival: Blues
team_name:  Ducks rival: Golden Knights
team_name:  Flames rival: Rangers
team_name:  Flyers rival: Ducks
team_name:  Golden Knights rival: Islanders
team_name:  Hurricanes rival: Rangers
team_name:  Islanders rival: Blackhawks
team_name:  Jets rival: Penguins
team_name:  Kings rival: Golden Knights
team_name:  Lightning rival: Golden Knights
team_name:  Maple Leafs rival: Sharks
team_name:  Oilers rival: Penguins
team_name:  Panthers rival: Kings
team_name:  Penguins rival: Blackhawks
team_name:  Predators rival: Red Wings
team_name:  Rangers rival: Islanders
team_name:  Red Wings rival: Lightning
team_name:  Sabres riva

### biggest_team_blowout
biggest diff between team and opponent goals in WIN for given team

In [49]:
data = full_df[full_df.won == True].groupby('teamName')['margin'].max()

In [53]:
for df,name in zip(dfs,df_names):
    print(name)
    print(df[df.won == True].groupby('teamName')['margin'].max())
    
    print("\n\n")

full
teamName
Avalanche          6
Blackhawks         9
Blue Jackets      10
Blues              7
Bruins             6
Canadiens          9
Canucks            6
Capitals           7
Coyotes            6
Devils             6
Ducks              8
Flames             7
Flyers             7
Golden Knights     7
Hurricanes         5
Islanders          7
Jets               6
Kings              7
Lightning          6
Maple Leafs        7
Oilers             7
Panthers           7
Penguins           7
Predators          7
Rangers            6
Red Wings          6
Sabres             5
Senators           6
Sharks             7
Stars              6
Wild               7
Name: margin, dtype: int64



small
teamName
Blackhawks    3
Bruins        5
Canadiens     2
Rangers       1
Red Wings     3
Senators      5
Name: margin, dtype: int64



vs
teamName
Bruins    3
Name: margin, dtype: int64





### worst_loss
biggest LOSS blowout

In [55]:
for df,name in zip(dfs,df_names):
    print(name)
    print(df[df.won == False].groupby('teamName')['margin'].min())
    
    print("\n\n")

full
teamName
Avalanche         -9
Blackhawks        -7
Blue Jackets      -7
Blues             -6
Bruins            -7
Canadiens        -10
Canucks           -8
Capitals          -6
Coyotes           -7
Devils            -6
Ducks             -7
Flames            -6
Flyers            -7
Golden Knights    -6
Hurricanes        -7
Islanders         -7
Jets              -7
Kings             -5
Lightning         -5
Maple Leafs       -7
Oilers            -7
Panthers          -7
Penguins          -9
Predators         -6
Rangers           -7
Red Wings         -9
Sabres            -7
Senators          -7
Sharks            -7
Stars             -7
Wild              -6
Name: margin, dtype: int64



small
teamName
Blackhawks   -3
Bruins       -1
Canadiens    -5
Penguins     -5
Rangers      -3
Red Wings    -3
Senators     -2
Name: margin, dtype: int64



vs
teamName
Rangers   -3
Name: margin, dtype: int64





### head_to_head
record (as hash-- win/loss) against specific opponent

In [67]:
for df,name in zip(dfs,df_names):
    print(name)
    
    data = df.groupby(['teamName','opp_team'])['won'].agg([len,sum])
    data['wins'] = data['sum']
    data['losses'] = data['len'] - data['sum']

    print(data[['wins','losses']].T.to_dict())
    
    print("\n\n")

full
{('Avalanche', 'Blackhawks'): {'wins': 14.0, 'losses': 12.0}, ('Avalanche', 'Blue Jackets'): {'wins': 6.0, 'losses': 7.0}, ('Avalanche', 'Blues'): {'wins': 10.0, 'losses': 17.0}, ('Avalanche', 'Bruins'): {'wins': 7.0, 'losses': 3.0}, ('Avalanche', 'Canadiens'): {'wins': 5.0, 'losses': 5.0}, ('Avalanche', 'Canucks'): {'wins': 8.0, 'losses': 11.0}, ('Avalanche', 'Capitals'): {'wins': 3.0, 'losses': 7.0}, ('Avalanche', 'Coyotes'): {'wins': 10.0, 'losses': 8.0}, ('Avalanche', 'Devils'): {'wins': 7.0, 'losses': 3.0}, ('Avalanche', 'Ducks'): {'wins': 7.0, 'losses': 11.0}, ('Avalanche', 'Flames'): {'wins': 8.0, 'losses': 11.0}, ('Avalanche', 'Flyers'): {'wins': 4.0, 'losses': 6.0}, ('Avalanche', 'Golden Knights'): {'wins': 1.0, 'losses': 2.0}, ('Avalanche', 'Hurricanes'): {'wins': 5.0, 'losses': 5.0}, ('Avalanche', 'Islanders'): {'wins': 5.0, 'losses': 5.0}, ('Avalanche', 'Jets'): {'wins': 12.0, 'losses': 12.0}, ('Avalanche', 'Kings'): {'wins': 5.0, 'losses': 13.0}, ('Avalanche', 'Lightn

### seasonal_summary

for each season that team has played, has hash with two keys "preseason" and "postseason" pointing to hash with win_percentage, total_goals_scored, total_goals_against, average_goals_scored, averate_goals_against

In [12]:
for df,name in zip(dfs,df_names):
    print(name)
    gb = df.groupby(['teamName','season',"type"])
    wp = pd.DataFrame(gb['won'].mean())
    wp.columns = ['win_pct']

    gol = gb['goals'].agg([sum,np.mean])
    gol.columns = ["tot_goals","avg_goals"]

    ga = gb['opp_goal'].agg([sum,np.mean])
    ga.columns = ["tot_ga","gaa"]
    ga

    print(pd.concat([wp,gol,ga], axis = 1))
    print("\n\n")

full
                             win_pct  tot_goals  avg_goals  tot_ga       gaa
teamName     season   type                                                  
Avalanche    20122013 R     0.333333        114   2.375000     152  3.166667
             20132014 P     0.428571         20   2.857143      22  3.142857
                      R     0.634146        245   2.987805     220  2.682927
             20142015 R     0.475610        209   2.548780     227  2.768293
             20152016 R     0.475610        212   2.585366     240  2.926829
             20162017 R     0.268293        165   2.012195     278  3.390244
             20172018 P     0.333333         15   2.500000      22  3.666667
                      R     0.524390        255   3.109756     237  2.890244
Blackhawks   20122013 P     0.695652         64   2.782609      48  2.086957
                      R     0.750000        149   3.104167     102  2.125000
             20132014 P     0.578947         58   3.052632      55  2.8

# Season Statistics
### biggest_bust

biggest decrease between pre-season and regular season win pct  

ONly possible on full data -- no regular season data in small and very_small data

In [35]:

print("full")
data = full_df.groupby(['teamName','season','type'])['won'].mean()
to_df = []
for idx in data.index:
    to_df.append([*idx, data[idx]])
data = pd.DataFrame(to_df, columns = ["teamName","season","type","wp"])
p_team= p_season= p_gtype= p_wp = 0
diffs = []
for idx in data.index:

    team, season, g_type, wp = data.loc[idx,:]
    if team == p_team:
        if season == p_season:
            if g_type == "P":
                post_wp = wp
                reg_wp = p_wp
            else:
                post_wp = p_wp
                reg_wp = wp

            diffs.append([team, season, post_wp-reg_wp])
    p_team = team; p_season = season; p_gtype = g_type; p_wp = wp

aggd = pd.DataFrame(diffs, columns = ['team','season', 'post_minus_reg_win_pct'])
gb = aggd.groupby(['season', 'team'])['post_minus_reg_win_pct'].max().unstack().T

for col in gb:
    sort_pct = gb[col].sort_values(ascending= True)
    print("Season:", col, "Biggest bust team", sort_pct.index[0])

print("\n\n")


full
Season: 20122013 Biggest bust team Canucks
Season: 20132014 Biggest bust team Lightning
Season: 20142015 Biggest bust team Jets
Season: 20152016 Biggest bust team Kings
Season: 20162017 Biggest bust team Blackhawks
Season: 20172018 Biggest bust team Kings





### biggest_surprise
biggest increase between pre-season and regular season

In [36]:
print("full")
data = full_df.groupby(['teamName','season','type'])['won'].mean()
to_df = []
for idx in data.index:
    to_df.append([*idx, data[idx]])
data = pd.DataFrame(to_df, columns = ["teamName","season","type","wp"])
p_team= p_season= p_gtype= p_wp = 0
diffs = []
for idx in data.index:

    team, season, g_type, wp = data.loc[idx,:]
    if team == p_team:
        if season == p_season:
            if g_type == "P":
                post_wp = wp
                reg_wp = p_wp
            else:
                post_wp = p_wp
                reg_wp = wp

            diffs.append([team, season, post_wp-reg_wp])
    p_team = team; p_season = season; p_gtype = g_type; p_wp = wp

aggd = pd.DataFrame(diffs, columns = ['team','season', 'post_minus_reg_win_pct'])
gb = aggd.groupby(['season', 'team'])['post_minus_reg_win_pct'].max().unstack().T

for col in gb:
    sort_pct = gb[col].sort_values(ascending= False)
    print("Season:", col, "Biggest surprise team", sort_pct.index[0])

print("\n\n")

full
Season: 20122013 Biggest surprise team Sharks
Season: 20132014 Biggest surprise team Kings
Season: 20142015 Biggest surprise team Blackhawks
Season: 20152016 Biggest surprise team Lightning
Season: 20162017 Biggest surprise team Predators
Season: 20172018 Biggest surprise team Capitals





### winningest_coach
coach with best win pct for the season

In [41]:
for df,name in zip(dfs,df_names):
    print(name)
    data = df.groupby(['season','head_coach'])['won'].mean().unstack().T

    for col in data:
        print("season: ", col, "winingest_coach: ", data[col].sort_values(ascending = False).index[0])
        
    print("\n\n")

full
season:  20122013 winingest_coach:  Dan Lacroix
season:  20132014 winingest_coach:  Claude Julien
season:  20142015 winingest_coach:  Alain Vigneault
season:  20152016 winingest_coach:  Barry Trotz
season:  20162017 winingest_coach:  Barry Trotz
season:  20172018 winingest_coach:  Jon Cooper



small
season:  20122013 winingest_coach:  Claude Julien



vs
season:  20122013 winingest_coach:  Claude Julien





### worst_coach
worst win pct for season

In [42]:
for df,name in zip(dfs,df_names):
    print(name)
    data = df.groupby(['season','head_coach'])['won'].mean().unstack().T

    for col in data:
        print("season: ", col, "worst_coach: ", data[col].sort_values(ascending = True).index[0])
        
    print("\n\n")

full
season:  20122013 worst_coach:  Martin Raymond
season:  20132014 worst_coach:  Peter Laviolette
season:  20142015 worst_coach:  Craig MacTavish
season:  20152016 worst_coach:  Todd Richards
season:  20162017 worst_coach:  Jared Bednar
season:  20172018 worst_coach:  Phil Housley



small
season:  20122013 worst_coach:  Dan Bylsma



vs
season:  20122013 worst_coach:  John Tortorella





### most accurate_team
team with best ratio of shots to goal for season

In [51]:
for df,name in zip(dfs,df_names):
    print(name)
    data = df.groupby(['season','teamName'])[['shots','goals']].sum()
    data['shoot_pct'] = data['goals']/data['shots']
    us = data['shoot_pct'].unstack().T

    for col in us:
        print("season: ", col, "best_shooting: ", us[col].sort_values(ascending = False).index[0])

    print("\n\n")

full
season:  20122013 best_shooting:  Lightning
season:  20132014 best_shooting:  Ducks
season:  20142015 best_shooting:  Flames
season:  20152016 best_shooting:  Stars
season:  20162017 best_shooting:  Penguins
season:  20172018 best_shooting:  Capitals



small
season:  20122013 best_shooting:  Senators



vs
season:  20122013 best_shooting:  Bruins





### least_accurate_team
team with worst ratio of shots to goals for season

In [52]:
for df,name in zip(dfs,df_names):
    print(name)
    data = df.groupby(['season','teamName'])[['shots','goals']].sum()
    data['shoot_pct'] = data['goals']/data['shots']
    us = data['shoot_pct'].unstack().T

    for col in us:
        print("season: ", col, "worst_shooting: ", us[col].sort_values(ascending = True).index[0])

    print("\n\n")

full
season:  20122013 worst_shooting:  Senators
season:  20132014 worst_shooting:  Sabres
season:  20142015 worst_shooting:  Coyotes
season:  20152016 worst_shooting:  Maple Leafs
season:  20162017 worst_shooting:  Avalanche
season:  20172018 worst_shooting:  Canadiens



small
season:  20122013 worst_shooting:  Penguins



vs
season:  20122013 worst_shooting:  Rangers





### most_hits

team with most hits in season

In [54]:
for df,name in zip(dfs,df_names):
    print(name)
    us = df.groupby(['season','teamName'])['hits'].sum().unstack().T


    for col in us:
        print("season: ", col, "most_hits: ", us[col].sort_values(ascending = False).index[0])

    print("\n\n")

full
season:  20122013 most_hits:  Kings
season:  20132014 most_hits:  Kings
season:  20142015 most_hits:  Islanders
season:  20152016 most_hits:  Islanders
season:  20162017 most_hits:  Penguins
season:  20172018 most_hits:  Capitals



small
season:  20122013 most_hits:  Bruins



vs
season:  20122013 most_hits:  Bruins





### least_hits
team with least hits in season

In [53]:
for df,name in zip(dfs,df_names):
    print(name)
    us = df.groupby(['season','teamName'])['hits'].sum().unstack().T


    for col in us:
        print("season: ", col, "least_hits: ", us[col].sort_values(ascending = True).index[0])

    print("\n\n")

full
season:  20122013 least_hits:  Devils
season:  20132014 least_hits:  Devils
season:  20142015 least_hits:  Wild
season:  20152016 least_hits:  Canucks
season:  20162017 least_hits:  Blackhawks
season:  20172018 least_hits:  Blackhawks



small
season:  20122013 least_hits:  Penguins



vs
season:  20122013 least_hits:  Rangers





### power_play_goal_pct
pct of goals that were ppg for season (rounded to 100th)

In [58]:
gb = full_df.groupby("season")[['goals','powerPlayGoals']].sum()
gb['pct_ppg'] = gb['powerPlayGoals']/gb['goals']
gb['pct_ppg']

season
20122013    0.227315
20132014    0.219935
20142015    0.214521
20152016    0.219083
20162017    0.209283
20172018    0.212083
Name: pct_ppg, dtype: float64